### Scraping tool to get reviews from restaurants. Firstly the function scrapeRestaurantsUrlsAll is used to gether all the restaurant urls based on location. Then we iterate through those urls and collect all the reviews.  

### Keyrðu cellurnar þangað til þú sérð

In [1]:
import requests 
from bs4 import BeautifulSoup
import csv 
from selenium import webdriver
import time
import sys
import os
import argparse
import string
import pickle

In [2]:
# Add urls for all "next" pages
def scrapeRestaurantsUrls(tripURLs):
    urls =[]
    for url in tripURLs:
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
        results = soup.find('div', class_='_1kXteagE')
        stores = results.find_all('div', class_='wQjYiB7z')
        for store in stores:
            unModifiedUrl = str(store.find('a', href=True)['href'])
            urls.append('https://www.tripadvisor.com'+unModifiedUrl)
    return urls

# Add urls for all "next" pages
def scrapeRestaurantsUrlsAll(url, limit=100):
    store_name = []
    urls = []
    limit_set = 1
    nextPage = True
    while nextPage and limit_set <= limit:
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
        results = soup.find('div', class_='_1kXteagE')
        stores = results.find_all('div', class_='wQjYiB7z') 
        for store in stores:
            if store.find('a', class_ = '_15_ydu6b').text[0].isdigit(): # skip the ones that er sponsored since they will also come later.
                
                print(store.find('a', class_ = '_15_ydu6b').text)
                unModifiedUrl = str(store.find('a', href=True)['href'])
                urls.append('https://www.tripadvisor.com'+unModifiedUrl)
        limit_set += 1
        #Go to next page if exists
        try:
            print('tried next in finding all')
            unModifiedUrl = str(soup.find('a', class_ = 'nav next rndBtn ui_button primary taLnk',href=True)['href'])
            # print(unModifiedUrl, 'later unmod')
            url = 'https://www.tripadvisor.com' + unModifiedUrl
            # print('new url is ', url)
        except:
            print('no next in finding all')
            nextPage = False

    with open(pathAllRestaurants, 'wb') as f:
        pickle.dump(urls, f)

    print(f'Total restaurant count: {len(urls)}')
    return urls

def scrapeRestaurantInfo(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    storeName = soup.find('h1', class_='_3a1XQ88S').text
    avgRating = soup.find('span', class_='r2Cf69qf').text.strip()
    storeAddress = soup.find('div', class_= '_2vbD36Hr _36TL14Jn').find('span', class_='_2saB_OSe').text.strip()
#     urlAddress = str(soup.find('div', class_ = '_2vbD36Hr _36TL14Jn').find('span').find('a', href=True)['href'])
    nrReviews = soup.find('a', class_='_10Iv7dOs').text.strip().split()[0]
    cousineType = [word.text for  word in soup.find('span', class_='_13OzAOXO _34GKdBMV').find_all('a')]
    nrPos = soup.find('a', class_='_15QfMZ2L').find('b').find('span').text.strip()
    with open(pathToStoreInfo, mode='a', encoding="utf-8") as trip:
        data_writer = csv.writer(trip, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
        data_writer.writerow([storeName, storeAddress, avgRating, nrReviews, cousineType[0], cousineType[1:], nrPos])

In [3]:
# driver = webdriver.Chrome('chromedriver.exe')
# This should be set to the path of the folder holding the python script.
    
def get_reviews(url):
    print(url)
    #if you want to scrape restaurants info

#     scrapeRestaurantInfo(url)

    nextPage = True
    while nextPage:
        #Requests
        driver.get(url)
        time.sleep(1)
        #Click More button
#         more = driver.find_elements_by_xpath("//span[contains(text(),'More')]")
        more = driver.find_elements_by_xpath("//span[@class='taLnk ulBlueLinks'][contains(text(),'More')]")
        # Push all buttons that unclude the "More" option on each review.
        for x in range(0,len(more)):
            try:
                driver.execute_script("arguments[0].click();", more[x])
                time.sleep(3)
            except:
                pass
            
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        #Store name
        storeName = soup.find('h1', class_='_3a1XQ88S').text
        #Reviews
        results = soup.find('div', class_='listContainer hide-more-mobile')
        try:
            reviews = results.find_all('div', class_='prw_rup prw_reviews_review_resp')
        except Exception:
            continue
        #Export to csv
        try:
            with open(pathToReviews, mode='a', encoding="utf-8") as trip_data:
                data_writer = csv.writer(trip_data, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
                for review in reviews:
                    ratingDate = review.find('span', class_='ratingDate').get('title')
                    reviewHeader = review.find('span', class_='noQuotes').text
                    text_review = review.find('p', class_='partial_entry')
                    if len(text_review.contents) > 2:
                        reviewText = str(text_review.contents[0][:-3]) + ' ' + str(text_review.contents[1].text)
                    else:
                        reviewText = text_review.text
                    reviewerUsername = review.find('div', class_='info_text pointer_cursor')
                    reviewerUsername = reviewerUsername.select('div > div')[0].get_text(strip=True)
                    rating = review.find('div', class_='ui_column is-9').findChildren('span')
                    rating = str(rating[0]).split('_')[3].split('0')[0]
                    data_writer.writerow([storeName, reviewerUsername, ratingDate, reviewHeader, reviewText, rating])
        except:
            pass

        #Go to next page if exists
        try:
            unModifiedUrl = str(soup.find('div', class_ = 'prw_rup prw_common_responsive_pagination').find('a', class_='nav next ui_button primary', href=True).get('href'))
            # unModifiedUrl = str(soup.find('a', class_ = 'nav next ui_button primary',href=True)['href'])
            url = 'https://www.tripadvisor.com' + unModifiedUrl
        except:
            nextPage = False


## INFO

Þú getur valið hversu mikið þú tekur í keyrslunni en ég held að 300 ætti alveg að vera fínt. 

Það er hægt að copy-a þessu notebook og keyra 2 samtímis þá breytiru bara 0:150 og 150:300 fyrir hvoru keyrsluna.

Ég sé svo um að eyða út fyrstu 300 línunum og skoða svo þau url sem voru með vesen "Save-a url með vesen". Þú mátt runna þá cellu einnig svo vandræðaslóðirnar séu vistaðar.

In [4]:
# Create new reviews.csv path for run
pathToReviews = "reviews6_kristin.csv"

with open(pathToReviews, mode='a', encoding="utf-8") as trip_data:
    data_writer = csv.writer(trip_data, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
    data_writer.writerow(['storeName', 'reviewerUsername', 'ratingDate', 'reviewHeader','reviewText', 'rating'])

In [5]:
# Þetta er aðalskjalið sem heldur utan um þá linka sem eru eftir.
with open("urls_left.txt", "rb") as f:   # Unpickling
    urls_left = pickle.load(f)

driver_path = f'{os.getcwd()}/chromedriver' # Get driver to run selenium
driver = webdriver.Chrome(driver_path)

finished = []
bad_url = []
rest_300 = urls_left[300:500]
for url in rest_300:
    try:
        get_reviews(url)
        finished.append(url)
    except:
        bad_url.append(url)

https://www.tripadvisor.com/Restaurant_Review-g189541-d17450924-Reviews-Met_Cph-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d14141418-Reviews-Glod-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d18338046-Reviews-Osteria_16-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d13288611-Reviews-The_Storm_Inn-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d14969415-Reviews-Victorious_Sportsbar_Cafe-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d787443-Reviews-Pier_5-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g806262-d3589515-Reviews-Hos_Trine_og_Christian-Frederiksberg_Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d8019133-Reviews-Plenum-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d5486027-Reviews-Bevi_Bevi-Copenhagen_Zealand.html
https://www.trip

https://www.tripadvisor.com/Restaurant_Review-g189541-d7360435-Reviews-The_Scottish_Pub-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d3434703-Reviews-Brasserie_Storm-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d12034042-Reviews-Restauranten_i_Koncerthuset-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d13079625-Reviews-Maui-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d806716-Reviews-Le_Basilic-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g806262-d1504892-Reviews-Omakase-Frederiksberg_Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d4298254-Reviews-Svane_Pub-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d8659321-Reviews-Osterbros_Originale_Burgerrestaurant-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d4131775-Reviews-Faerge_Cafeen-Copenhag

https://www.tripadvisor.com/Restaurant_Review-g806262-d1149422-Reviews-VIET_NAM_NAM-Frederiksberg_Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d8683929-Reviews-Haps-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g806262-d10215093-Reviews-Cafe_Vivaldi_Falkoner_Alle-Frederiksberg_Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d14993385-Reviews-Alice_Ice_Cream_and_Coffee-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d9770109-Reviews-Gourmet_street_food_n_soup-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d1119513-Reviews-Charlie_Scotts_Bar-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g189541-d20110260-Reviews-Neighbourhood-Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g806262-d12438576-Reviews-CORK_Vinbar-Frederiksberg_Copenhagen_Zealand.html
https://www.tripadvisor.com/Restaurant_Review-g806

## Save-a url með vesen

In [7]:
# For curiosity how many urls didn't get scraped
print(len(bad_url))

0


In [6]:
# write out the urls that are left to scrape
with open('bad_url2.txt', 'wb') as f:
    pickle.dump(bad_url, f)

In [8]:
for row in bad_url:
    print(row)